In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
train

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold

# 將train.csv 讀入pandas的DataFrame。
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
# 從train取出圖像資料，放於DataFrame object。
train_x = train.drop(['label'], axis=1)
# 從train取出正確答案標籤，放於Series object。
train_y = train['label']
# 將test.csv 讀入 pandas的DataFrame。
test_x = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

#將 train 的資料分為4個部分，其中3個用於訓練、1個用於驗證。
kf = KFold(n_splits=4, shuffle=True, random_state=123)
# 取得用於訓練、以及用於驗證的列的索引。
tr_idx, va_idx = list(kf.split(train_x))[0]
# 分別取得訓練圖像資料與驗證圖像資料以及正確答案標籤
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# 將圖像的像素值除以255.0、使像素值落在0 ~ 1.0範圍內，變換為numpy.array。
tr_x, va_x = np.array(tr_x / 255.0), np.array(va_x / 255.0)

# 將正確答案（標籤）變換為One-hot encoding
tr_y = to_categorical(tr_y, 10) # numpy.ndarray object
va_y = to_categorical(va_y, 10) # numpy.ndarray object

#輸出 tr_x、tr_y、va_x、va_y 的形狀。
print(tr_x.shape)
print(tr_y.shape)
print(va_x.shape)
print(va_y.shape)


In [ ]:
from collections import Counter
count = Counter(train['label']) # 確認0 ~ 9各有幾張。
count


In [ ]:
import seaborn as sns
sns.countplot(train['label'] ) # 將0 ~ 9的張數製成圖表。


In [ ]:
print(tr_x[0]) #輸出訓練資料的第1列。

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# 從訓練資料當中取出50張，進行繪製。
plt.figure(figsize=(12,10))
x, y = 10, 5 # 輸出10欄5列。
for i in range(50):
    plt.subplot(y, x, i+1)
    # 描繪為28 × 28 的尺寸。
    plt.imshow(tr_x[i].reshape((28,28)),interpolation='nearest')
plt.show()

In [ ]:
# 建立神經網路
# 從tensorflow.keras.models 導入 Sequential
from tensorflow.keras.models import Sequential
# 從tensorflow.keras.layers 導入 Dense、Activation
from tensorflow.keras.layers import Dense, Activation

# 生成Sequential object。
model = Sequential()
# 第1層(隱藏層)
model.add(Dense(128,                      # 神經元數量。
                input_dim=tr_x.shape[1],  # 指定輸入的資料之形狀。
                activation='sigmoid'      # 激活函數為Sigmoid。
               ))
model.add(Dense(
    10,                # 神經元的數量相同，為10。
    activation='softmax'  # 指定要適合用於多元分類的Softmax。
))
model.compile(
    # 使用交叉熵誤差函數作為損失函數。
    loss='categorical_crossentropy',
    # 優化器為Adam。
    optimizer='adam',
    # 指定準確率做為學習評價。
    metrics=['accuracy']
)
# 輸出模型的結構。
model.summary()


In [ ]:
result = model.fit(tr_x, tr_y,                  # 訓練資料與正確答案標籤。
                   epochs=5,                    # 學習次數為5次。
                   batch_size=100,              # 小批次的尺寸為100。
                   validation_data=(va_x, va_y),# 制定用於驗證的資料。
                   verbose=1)                   # 輸出學習進度。


In [ ]:
# 使用測試資料進行預測、並將結果帶入NumPy 序列。
result = model.predict(test_x)


In [ ]:
# 輸出前5個預測結果。
print(result[:5])
# 輸出最大值的索引（所預測的數字）。
print([x.argmax() for x in result[:5]])
# 將預測出的數字帶入NumPy 序列。
y_test = [x.argmax() for x in result]


In [ ]:
submit_df = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
# 輸出前5列。
submit_df.head()


In [ ]:
submit_df['Label'] = y_test
# 輸出前5列。
submit_df.head()


In [ ]:
submit_df.to_csv('submission.csv', index=False)